In [1]:
class Model:
    def __init__(self,sess,name):
        self.sess=sess
        self.name=name
        self._build_net()
    def _build_net(self):#변수 설정 및 network 구성
        with tf.variable_scope(self.name):
            self.learning_rate=0.001
            
            self.X=tf.placeholder(tf.float32,shape=[None,784])
            self.X_img = tf.reshape(self.X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
            self.Y = tf.placeholder(tf.float32, [None, 10])
            self.training = tf.placeholder(tf.bool)
            #cnn network
            self.L1=tf.layers.conv2d(inputs=self.X_img,filters=32,kernel_size=[3,3],padding="SAME",activation=tf.nn.relu)
            self.L1=tf.layers.max_pooling2d(inputs=self.L1,pool_size=[2,2],padding="SAME",strides=2)
            self.L1_drop=tf.layers.dropout(inputs=self.L1,rate=0.7,training=self.training)
            
            self.L2=tf.layers.conv2d(inputs=self.L1_drop,filters=64,kernel_size=[3,3],padding="SAME",activation=tf.nn.relu)
            self.L2=tf.layers.max_pooling2d(inputs=self.L2,pool_size=[2,2],padding="SAME",strides=2)
            self.L2_drop=tf.layers.dropout(inputs=self.L2,rate=0.7,training=self.training)
            
            self.L3=tf.layers.conv2d(inputs=self.L2_drop,filters=128,kernel_size=[3,3],padding="SAME",activation=tf.nn.relu)
            self.L3=tf.layers.max_pooling2d(inputs=self.L3,pool_size=[2,2],padding="SAME",strides=2)
            self.L3_drop=tf.layers.dropout(inputs=self.L3,rate=0.7,training=self.training)
            
            self.flat=tf.reshape(self.L3_drop,[-1,4*4*128])
            #fully connected network
            self.FL1=tf.layers.dense(inputs=self.flat,units=625,activation=tf.nn.relu)
            self.FL1_drop=tf.layers.dropout(self.FL1,0.7,training=self.training)
            
            self.logits=tf.layers.dense(inputs=self.FL1_drop,units=10)
            
            #cost & optimizer
            self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
            #accuracy & prediction
            self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
    def predicted(self,x_test,training=False):
        return self.sess.run(self.logits,feed_dict={self.X:x_test,self.training:training})
    def get_accuracy(self,x_test,y_test,training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.training:training})
    def train(self,x_data,y_data,training=True):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data,self.training:training})

In [2]:
import tensorflow as tf
import numpy as np
import random
# import matplotlib.pyplot as plt
import time
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility
#load mnist data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess=tf.Session()
models=[]
num_models=10

training_epochs = 15
batch_size = 100

for m in range(num_models):
    models.append(Model(sess,"model"+str(m)))
sess.run(tf.global_variables_initializer())
start_time=time.time()

# train my model
print('Learning started. It takes sometime.')
# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys =mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:','%04d'%(epoch + 1),'cost =', avg_cost_list)

print('Learning Finished!')
print('learning time : ',time.time()-start_time)



C:\Users\Robotmedia9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = [0.97646909 0.89665843 0.9136703  0.91228561 0.9686816  0.899685
 0.8631625  0.91038239 0.94287076 0.88138836]
Epoch: 0002 cost = [0.34195169 0.33731705 0.33734675 0.33603003 0.34763766 0.33072022
 0.33743168 0.33806542 0.34829367 0.33044027]
Epoch: 0003 cost = [0.26370975 0.2676558  0.26828072 0.26822135 0.2691241  0.26084358
 0.26280524 0.26650742 0.26981815 0.26701984]
Epoch: 0004 cost = [0.23305772 0.23314818 0.23620417 0.24086489 0.23605815 0.23364954
 0.23451815 0.23039156 0.23831221 0.23369453]
Epoch: 0005 cost = [0.20835859 0.21424956 0.21807894 0.21436056 0.21464681 0.20806126
 0.21481723 0.21076636 0.21667431 0.21413873]
Epoch: 0006 cost = [0.19899496 0.20206501 0.20201499 0.20320623 0.20763668 0.199385
 0.20112415 0.20173814 0.20

In [3]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predicted(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

0 Accuracy: 0.9879
1 Accuracy: 0.988
2 Accuracy: 0.9881
3 Accuracy: 0.9884
4 Accuracy: 0.989
5 Accuracy: 0.9876
6 Accuracy: 0.9882
7 Accuracy: 0.9889
8 Accuracy: 0.988
9 Accuracy: 0.9874
Ensemble accuracy: 0.9893
